In [42]:
from logicqubit.logic import *
from math import *
import random
import operator
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex

In [43]:
def print_matrix(matrix):
    value = sp.latex(sp.Matrix(matrix))
    display(Math(value))

In [44]:
logicQuBit = LogicQuBit(2)

B = Qubit()
A = Qubit()

# generate entangled state |01>-|10>
A.H()
B.CNOT(A)
B.X()
B.Z()

psi = logicQuBit.getPsi()

# Axes
gates = Gates()
M = {}
M["ab"] = gates.Z().kron((gates.Z()+gates.X())/sqrt(2))   # Z and (Z+X)/sqrt(2)
M["a'b"] = gates.X().kron((gates.Z()+gates.X())/sqrt(2))  # X and (Z+X)/sqrt(2)
M["ab'"] = gates.Z().kron((gates.Z()-gates.X())/sqrt(2))  # Z and (Z-X)/sqrt(2)
M["a'b'"] = gates.X().kron((gates.Z()-gates.X())/sqrt(2)) # X and (Z-X)/sqrt(2)

# Expected values for ab, a'b, ab' and a'b'
values = {}
values["ab"] = (psi.adjoint()*M["ab"]*psi).get()[0]
values["a'b"] = (psi.adjoint()*M["a'b"]*psi).get()[0]
values["ab'"] = (psi.adjoint()*M["ab'"]*psi).get()[0]
values["a'b'"] = (psi.adjoint()*M["a'b'"]*psi).get()[0]

print(logicQuBit.getDictPsi())
print(values)

# Bell inequality value
inequality = values["ab"] + values["a'b"] + values["ab'"] - values["a'b'"]
inequality = sqrt(inequality.real * inequality.real)
print("Inequality: "+str(inequality))

{'00': 0j, '01': (0.7071067811865475+0j), '10': (-0.7071067811865475+0j), '11': 0j}
{'ab': array([-0.70710678+0.j]), "a'b": array([-0.70710678+0.j]), "ab'": array([-0.70710678+0.j]), "a'b'": array([0.70710678+0.j])}
Inequality: 2.8284271247461894


In [45]:
def measurement(type_A, type_B):
    logicQuBit = LogicQuBit(2)
    
    gates = Gates()
    ZZ = gates.Z().kron(gates.Z())
    
    A = Qubit()
    B = Qubit()

    # generate entangled state |01>-|10>
    A.H()
    B.CNOT(A)
    B.X()
    B.Z()

    # Alice choose the measurement
    if "a'" == type_A:
        A.H()

    # Bob choose the measurement
    if "b" == type_B:
        B.RY(-pi/4)
    elif "b'" == type_B:
        B.RY(pi/4)
        
    psi = logicQuBit.getPsi()
    value = (psi.adjoint()*ZZ*psi).get()[0]
    return value

values = {}
values["ab"] = measurement("a","b")
values["a'b"] = measurement("a'","b")
values["ab'"] = measurement("a","b'")
values["a'b'"] = measurement("a'","b'")
inequality = values["ab"] + values["a'b"] + values["ab'"] - values["a'b'"]
inequality = sqrt(inequality.real * inequality.real)
print("Inequality: "+str(inequality))

Inequality: 2.828427124746189


In [46]:
#print_matrix(((gates.Z()+gates.X())/sqrt(2)).get())
#print_matrix(gates.U3(1,pi/2,0,pi).get())
#print_matrix(gates.H().get())

In [50]:
# |<C>| = |<a.b> + <a'.b> + <a.b'> - <a'.b'>| <= 2
def bell_inequality(A, B):
    direction = {"ab":0, "a'b":0, "ab'":0, "a'b'":0}
    count = {"ab":0, "a'b":0, "ab'":0, "a'b'":0}
    
    # Expected values for ab, a'b, ab' and a'b'
    for a,b in zip(A,B):
        if a[0]=="a" and b[0]=="b":
            direction["ab"] += a[1]*b[1]
            count["ab"] += 1
        elif a[0]=="a'" and b[0]=="b":
            direction["a'b"] += a[1]*b[1]
            count["a'b"] += 1
        elif a[0]=="a" and b[0]=="b'":
            direction["ab'"] += a[1]*b[1]
            count["ab'"] += 1
        elif a[0]=="a'" and b[0]=="b'":
            direction["a'b'"] += a[1]*b[1]
            count["a'b'"] += 1

    direction["ab"] /= count["ab"]
    direction["a'b"] /= count["a'b"]
    direction["ab'"] /= count["ab'"]
    direction["a'b'"] /= count["a'b'"]
    # Bell inequality value
    inequality = direction["ab"] + direction["a'b"] + direction["ab'"] - direction["a'b'"]
    inequality = sqrt(inequality * inequality)
    print(direction)
    return inequality

def measurements(eva=False, attempts=10):
    to_minus_one = lambda x: -1 if x == 0 else x
    measure_A = []
    measure_B = []
  
    for i in range(attempts):
        logicQuBit = LogicQuBit(2)

        A = Qubit()
        B = Qubit()

        # generate entangled state |01>-|10>
        A.H()
        B.CNOT(A)
        B.X()
        B.Z()

        # Alice choose the measurement
        measure_type_A = random.choice(["a", "a'"])
        if "a'" == measure_type_A: # for x
            A.H()
            
        # Alice performs the measurement
        m_A = logicQuBit.Measure_One(A)[0]
        m_A = to_minus_one(m_A)

        # Eva performs the measurement
        if eva:
            measure_type_E = random.choice(["b", "b'"])
            if "b" == measure_type_E:
                B.RY(-pi/4)
            elif "b'" == measure_type_E:
                B.RY(pi/4)
            m_E = logicQuBit.Measure_One(B)[0]

        # Bob choose the measurement
        measure_type_B = random.choice(["b", "b'"])
        if "b" == measure_type_B:
            B.RY(-pi/4)
        elif "b'" == measure_type_B:
            B.RY(pi/4)
       
        # Bob performs the measurement
        m_B = logicQuBit.Measure_One(B)[0]
        m_B = to_minus_one(m_B)

        measure_A.append([measure_type_A, m_A])
        measure_B.append([measure_type_B, m_B])

    return measure_A, measure_B

measure_A, measure_B = measurements(eva=False, attempts=500)
bell = bell_inequality(measure_A, measure_B)
print(bell)

{'ab': -0.7522123893805309, "a'b": -0.6825396825396826, "ab'": -0.728, "a'b'": 0.7647058823529411}
2.9274579542731543


In [51]:
# without eva
measure_A, measure_B = measurements(eva=False, attempts=200)
bell = bell_inequality(measure_A, measure_B)
print(bell)

{'ab': -0.75, "a'b": -0.7446808510638298, "ab'": -0.7619047619047619, "a'b'": 0.6}
2.856585612968592


In [52]:
# with eva
measure_A, measure_B = measurements(eva=True, attempts=200)
bell = bell_inequality(measure_A, measure_B)
print(bell)

{'ab': -0.5238095238095238, "a'b": 0.125, "ab'": -0.42857142857142855, "a'b'": 0.037037037037037035}
0.8644179894179893
